##  3.2 UM-Bridge Integration for the target function

In this section, we show how to define a custom target function as an UM-Bridge model. The goal is to expose a probability distribution (posterior distribution) via the UM-Bridge framework, which can be used in MCMC algorithms like Metropolis-Hastings for sampling. We use a **log-normal distribution** because of its stability when working with logarithmic transformations, which avoids numerical instabilities in the acceptance probability calculation.

In the following code, we implement a log-posterior distribution using a 2-dimensional normal distribution with a predefined mean and covariance matrix. The model receives input values, evaluates the log of the posterior probability density function (log-pdf), and returns the result.

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
import umbridge
import nest_asyncio
nest_asyncio.apply()

class LogTargetDistributionModel(umbridge.Model):
    def __init__(self):
        super().__init__("posterior")

    def get_input_sizes(self, config):
        return [2] # 2D input

    def get_output_sizes(self, config):
        return [1] # 1D output log density

    def __call__(self, parameters, config):
        x = np.array(parameters[0])
        mean = np.array([0, 0])
        cov = np.array([[1, 0.5], [0.5, 1]])
        posterior = multivariate_normal.logpdf(x, mean, cov)
        return [[posterior]]

    def supports_evaluate(self):
        return True
    
# Create an instance of the model
model = LogTargetDistributionModel()

# Starting the server
umbridge.serve_models([model], 4242)

======== Running on http://0.0.0.0:4242 ========
(Press CTRL+C to quit)


### Next Steps
Once the server is running, clients can connect to it and use this model for their MCMC sampling tasks. For example, you can implement a Metropolis-Hastings client that queries this UM-Bridge server, evaluates the log-posterior at different points, and uses this information to generate samples from the target distribution.